<a href="https://colab.research.google.com/github/Au9u5tu-s/FaceNet-FaceRecognition/blob/master/Knowledge_graphs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import re
import pandas as pd
import bs4
import requests
import spacy
from spacy import displacy
nlp = spacy.load('en_core_web_sm')

from spacy.matcher import Matcher
from spacy.tokens import Span

import networkx as nx

import matplotlib.pyplot as plt
from tqdm import tqdm

In [ ]:
nlp = spacy.load('en_core_web_sm')

In [ ]:
df = pd.read_excel('/content/cleaned data.xlsx')

In [ ]:
df

,Headlines,Date,Price,prevclose
0,the guardian view on ryanair s model a union f...,2017-12-17,2690.16,2675.81
1,butchers carve out a niche as uk shoppers opt ...,2017-12-17,2690.16,2675.81
2,this year has been about companies and jobs wi...,2017-12-17,2690.16,2675.81
3,youngest staff to be given uk workplace pensio...,2017-12-17,2690.16,2675.81
4,paul dacre and pay how best to spend the mail ...,2017-12-17,2690.16,2675.81
...,...,...,...,...
53325,world bank calls on creditors to cut poorest n...,2020-07-18,3251.84,3224.73
53326,british airways retires boeing 747 fleet as co...,2020-07-18,3251.84,3224.73
53327,what will changes to england s lockdown rules ...,2020-07-18,3251.84,3224.73
53328,atol protection to be extended to vouchers on ...,2020-07-18,3251.84,3224.73


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53330 entries, 0 to 53329
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Headlines  53330 non-null  object        
 1   Date       53330 non-null  datetime64[ns]
 2   Price      53330 non-null  float64       
 3   prevclose  53330 non-null  float64       
dtypes: datetime64[ns](1), float64(2), object(1)
memory usage: 1.6+ MB


In [ ]:
df.describe()

,Price,prevclose
count,53330.000000,53330.000000
mean,2871.109031,2871.170248
std,206.322849,204.872230
min,2237.400000,2237.400000
25%,2730.200000,2730.200000
50%,2856.270000,2856.270000
75%,2995.680000,2993.070000
max,3386.150000,3386.150000


In [ ]:
def get_entities(sent):
    ent1 = ""
    ent2 = ""

    prv_tok_dep = ""
    prv_tok_text = ""

    prefix = ""
    modifier = ""
    for tok in nlp(sent):

        if tok.dep_ != "punct":
            if tok.dep_ == "compound":
                prefix = tok.text
                if prv_tok_dep == "compound":
                    prefix = prv_tok_text + " " + tok.text
            if tok.dep_.endswith("mod") == True:
                modifier = tok.text
                if prv_tok_dep == "compound":
                    modifier = prv_tok_text + " " + tok.text
            if tok.dep_.find("subj") == True:
                ent1 = modifier + " " + prefix + " " + tok.text
                prefix = ""
                modifier = ""
                prv_tok_dep = ""
                prv_tok_text = ""
            if tok.dep_.find("obj") == True:
                ent2 = modifier + " " + prefix + " " + tok.text
            prv_tok_dep = tok.dep_
            prv_tok_text = tok.text


    return [ent1.strip(), ent2.strip()]

In [ ]:
# for news in df['Headlines']:
#     sentence = nlp(news)
#     for word in sentence:
#       print(word.text, "...", word.dep_)
#     break

In [ ]:
entity_pairs = []

for i in tqdm(df["Headlines"]):
    entity_pairs.append(get_entities(i))

100%|██████████| 53330/53330 [07:08<00:00, 124.50it/s]


In [ ]:
entity_pairs

[['', 'guardian model'],
 ['uk shoppers', 'indie stores'],
 ['year', 'wages'],
 ['youngest  staff', 'qualifying age'],
 ['', 'best mail millions'],
 ['unilever', '£ household 6bn'],
 ['david cameron', '£ china investment initiative'],
 ['us mall operators', 'new e space'],
 ['global shopping centre giants', 'christmas buying spree'],
 ['', 'bad uk tech firms'],
 ['why  business', 'corbyn government'],
 ['britons', 'new gin bottles'],
 ['british pension scheme members', 'financial  firms'],
 ['going', 'furniture time'],
 ['trade secrets', 'foreign rivals filing'],
 ['', 'norwich factory site'],
 ['how  investing', 'brighter  africa'],
 ['', 'insurance downgrade'],
 ['', 'new fundraising trend'],
 ['alabama unions', 'organized doug labor'],
 ['basic  income', 'us'],
 ['brexit', 'british  publisher'],
 ['', '$ carmichael coalmine operation'],
 ['ryanair pilots', 'one day christmas'],
 ['', '21st  century'],
 ['lse  chairman', 'him'],
 ['epa rule change', 'mine cleanup critics'],
 ['', '']

In [ ]:
def get_relation(sent):

    doc = nlp(sent)

    # Matcher class object
    matcher = Matcher(nlp.vocab)

    #define the pattern
    pattern = [{'DEP':'ROOT'},
            {'DEP':'prep','OP':"?"},
            {'DEP':'agent','OP':"?"},
            {'POS':'ADJ','OP':"?"}]

    matcher.add("matching_1", [pattern])

    matches = matcher(doc)
    k = len(matches) - 1

    span = doc[matches[k][1]:matches[k][2]]

    return(span.text)

In [ ]:
relations = [get_relation(i) for i in tqdm(df['Headlines'])]

100%|██████████| 53330/53330 [07:34<00:00, 117.46it/s]


In [ ]:
pd.Series(relations).value_counts()[:50]

says           3455
is              695
s               351
sees            308
plans           299
seeks           285
say             269
hit             235
live            231
raises          208
hits            208
warns           202
faces           201
take            197
fall            188
cuts            181
facebook        180
urges           175
are             175
takes           174
be              171
cut             160
face            156
wins            148
have            141
rise            140
has             140
set             135
expects         134
remix           125
tells           124
won             120
aims            116
wants           115
asks            113
makes           113
gets            112
falls           112
extends         110
said on mad     107
rises           106
pay             102
make            101
need             99
agrees           94
offers           94
loses            93
considers        92
view on          91
plan             90


In [ ]:
# extract subject
source = [i[0] for i in entity_pairs]

# extract object
target = [i[1] for i in entity_pairs]

kg_df = pd.DataFrame({'source':source, 'target':target, 'edge':relations})
kg_df

,source,target,edge
0,,guardian model,view on
1,uk shoppers,indie stores,carve
2,year,wages,been about
3,youngest staff,qualifying age,given
4,,best mail millions,dacre
...,...,...,...
53325,world bank,poorest nations debt payments,calls on
53326,british airways,covid hits travel,boeing
53327,changes,lockdown me,mean for
53328,,19 atol cancellations,protection


In [ ]:
G=nx.from_pandas_edgelist(kg_df, "source", "target",
                          edge_attr=True, create_using=nx.MultiDiGraph())

In [ ]:
# plt.figure(figsize=(12,12))

# pos = nx.spring_layout(G)
# nx.draw(G, with_labels=True, node_color='red', edge_cmap=plt.cm.Blues, pos = pos)
# plt.show()

In [ ]:
# G=nx.from_pandas_edgelist(kg_df[kg_df['edge']=="wants"], "source", "target",
#                           edge_attr=True, create_using=nx.MultiDiGraph())

# plt.figure(figsize=(12,12))
# pos = nx.spring_layout(G, k = 0.5) # k regulates the distance between nodes
# nx.draw(G, with_labels=True, node_color='red', node_size=1500, edge_cmap=plt.cm.Blues, pos = pos, font_weight='bold')
# plt.show()

In [ ]:
merged_df = pd.concat((df,kg_df), axis = 1)
merged_df.to_csv('/content/Knowledgegraphs.csv', index="false")